In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets 
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [2]:
#标签和类别数进行对应
train = pd.read_csv("/kaggle/input/classify-leaves/train.csv")
labels = list(pd.read_csv("/kaggle/input/classify-leaves/train.csv")['label'])
labels_unique = list(set(list(labels))) #list index--labels
label_nums = []
for i in range(len(labels)):
    label_nums.append(labels_unique.index(labels[i]))
train['number'] = label_nums
train.to_csv("./train_num_label.csv", index = 0) #记录对应关系

In [3]:
class Leaf_Train_Dataset(Dataset):
    '''
    树叶数据集的训练集 自定义Dataset
    '''
    def __init__(self, train_path, transform = None):
        '''
        train_path : 传入记录图像路径及其标号的csv文件
        transform : 对图像进行的变换
        '''
        super().__init__()
        self.train_csv = pd.read_csv(train_path)
        self.image_path = list(self.train_csv['image']) #图像所在地址记录
        self.label_nums = list(self.train_csv['number']) #图像的标号记录
        self.transform = transform
    def __getitem__(self, idx):
        '''
        idx : 所需要获取的图像的索引
        return : image， label
        '''
        image = Image.open(os.path.join("/kaggle/input/classify-leaves", self.image_path[idx]))
        if(self.transform != None):
            image = self.transform(image)
        label = self.label_nums[idx]
        return image, label
    def __len__(self):
        return len(self.image_path)

class Leaf_Test_Dataset(Dataset):
    '''
    树叶数据集的训练集 自定义Dataset
    '''
    def __init__(self, test_path, transform = None):
        '''
        train_path : 传入记录图像路径及其标号的csv文件
        transform : 对图像进行的变换
        '''
        super().__init__()
        self.test_csv = pd.read_csv(test_path)
        self.image_path = list(self.test_csv['image']) #图像所在地址记录
        self.transform = transform
    def __getitem__(self, idx):
        '''
        idx : 所需要获取的图像的索引
        return : image
        '''
        image = Image.open(os.path.join("/kaggle/input/classify-leaves", self.image_path[idx]))
        if(self.transform != None):
            image = self.transform(image)
        return image
    def __len__(self):
        return len(self.image_path)

In [4]:
#data agumentation
transforms_train = torchvision.transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.5, 1),ratio=(3/4, 4/3)),
    transforms.ToTensor()
])
transforms_test = torchvision.transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [5]:
def train(train_loader, valid_loader, device = torch.device("cuda:0")):
    net = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
    in_features = net.fc.in_features
    net.fc = nn.Linear(in_features, 176)
    epoch = 5
    losses = []
    optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-3)
    loss = nn.CrossEntropyLoss(reduction='mean')
    for i in range(epoch):
        acc = 0
        loss_sum = 0
        for x, y in train_loader:
            net = net.to(device)
            x = torch.as_tensor(x, dtype=torch.float)
            x = x.to(device)
            y = y.to(device)
            y_hat = net(x)
            loss_temp = loss(y_hat, y)
            loss_sum += loss_temp
            optimizer.zero_grad()
            loss_temp.backward()
            optimizer.step()
            acc += torch.sum(y_hat.argmax(dim=1).type(y.dtype) == y)
        losses.append(loss_sum.cpu().detach().numpy() / len(train_loader))
        print( "epoch: ", i, "loss=", loss_sum.item(), "训练集准确度=",(acc/(len(train_loader)*train_loader.batch_size)).item(),end="")

#         test_acc = 0
#         for x, y in valid_loader:
#             x = x.to(device)
#             x = torch.as_tensor(x, dtype=torch.float)
#             y = y.to(device)
#             y_hat = net(x)
#             test_acc += torch.sum(y_hat.argmax(dim=1).type(y.dtype) == y)
#         print("验证集准确度", (test_acc / (len(valid_loader)*valid_loader.batch_size)).item())
    torch.save(net.state_dict(), 'resnet50')
    return net

In [6]:
trainset = Leaf_Train_Dataset("/kaggle/working/train_num_label.csv", transform = transforms_train)
testset = Leaf_Test_Dataset("/kaggle/input/classify-leaves/test.csv", transform = transforms_test)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, drop_last=False)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, drop_last=False)
net = train(train_loader, test_loader)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 89.1MB/s]


epoch:  0 loss= 1256.944091796875 训练集准确度= 0.13038980960845947epoch:  1 loss= 780.0076293945312 训练集准确度= 0.37086236476898193epoch:  2 loss= 557.8914184570312 训练集准确度= 0.5140461921691895epoch:  3 loss= 435.74066162109375 训练集准确度= 0.6126959919929504epoch:  4 loss= 358.3039855957031 训练集准确度= 0.6740527153015137

In [7]:
device = torch.device("cuda:0")
predict = torch.tensor([]).to(device)
with torch.no_grad():
    for x in test_loader:
        net = net.to(device)
        x = x.to(device)
        x = torch.as_tensor(x, dtype=torch.float)
        y_hat = net(x)
        predict = torch.cat((y_hat, predict), dim=0)
    predict = torch.argmax(predict, dim=1).reshape(-1)
predict_label = []
for i in range(predict.shape[0]):
    predict_label.append(labels_unique[predict[i]])

submission = pd.read_csv("/kaggle/input/classify-leaves/test.csv")
submission["label"] = pd.Series(predict_label)
submission.to_csv("result.csv", index=False)